# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fd1d8ffca00>
├── 'a' --> tensor([[-0.3744,  1.5878,  3.7638],
│                   [ 0.4636,  0.2477, -0.2154]])
└── 'x' --> <FastTreeValue 0x7fd1d8ffcf40>
    └── 'c' --> tensor([[ 0.8382,  0.8111, -1.2707,  0.0985],
                        [ 0.1601, -0.2383,  0.5786,  0.0639],
                        [-1.9713,  0.9861, -2.0437, -1.0594]])

In [4]:
t.a

tensor([[-0.3744,  1.5878,  3.7638],
        [ 0.4636,  0.2477, -0.2154]])

In [5]:
%timeit t.a

85.4 ns ± 0.0537 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fd1d8ffca00>
├── 'a' --> tensor([[-0.0357,  1.7343,  1.4026],
│                   [-1.5537, -0.1947,  0.8365]])
└── 'x' --> <FastTreeValue 0x7fd1d8ffcf40>
    └── 'c' --> tensor([[ 0.8382,  0.8111, -1.2707,  0.0985],
                        [ 0.1601, -0.2383,  0.5786,  0.0639],
                        [-1.9713,  0.9861, -2.0437, -1.0594]])

In [7]:
%timeit t.a = new_value

85.7 ns ± 0.0905 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.3744,  1.5878,  3.7638],
               [ 0.4636,  0.2477, -0.2154]]),
    x: Batch(
           c: tensor([[ 0.8382,  0.8111, -1.2707,  0.0985],
                      [ 0.1601, -0.2383,  0.5786,  0.0639],
                      [-1.9713,  0.9861, -2.0437, -1.0594]]),
       ),
)

In [10]:
b.a

tensor([[-0.3744,  1.5878,  3.7638],
        [ 0.4636,  0.2477, -0.2154]])

In [11]:
%timeit b.a

77.2 ns ± 0.0933 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.3733,  0.9931, -1.8552],
               [ 0.6453,  0.1023,  1.3317]]),
    x: Batch(
           c: tensor([[ 0.8382,  0.8111, -1.2707,  0.0985],
                      [ 0.1601, -0.2383,  0.5786,  0.0639],
                      [-1.9713,  0.9861, -2.0437, -1.0594]]),
       ),
)

In [13]:
%timeit b.a = new_value

648 ns ± 0.174 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.05 µs ± 0.236 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13 µs ± 21.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

173 µs ± 1.94 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

175 µs ± 1.04 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fd1d4f7a220>
├── 'a' --> tensor([[[-0.3744,  1.5878,  3.7638],
│                    [ 0.4636,  0.2477, -0.2154]],
│           
│                   [[-0.3744,  1.5878,  3.7638],
│                    [ 0.4636,  0.2477, -0.2154]],
│           
│                   [[-0.3744,  1.5878,  3.7638],
│                    [ 0.4636,  0.2477, -0.2154]],
│           
│                   [[-0.3744,  1.5878,  3.7638],
│                    [ 0.4636,  0.2477, -0.2154]],
│           
│                   [[-0.3744,  1.5878,  3.7638],
│                    [ 0.4636,  0.2477, -0.2154]],
│           
│                   [[-0.3744,  1.5878,  3.7638],
│                    [ 0.4636,  0.2477, -0.2154]],
│           
│                   [[-0.3744,  1.5878,  3.7638],
│                    [ 0.4636,  0.2477, -0.2154]],
│           
│                   [[-0.3744,  1.5878,  3.7638],
│                    [ 0.4636,  0.2477, -0.2154]]])
└── 'x' --> <FastTreeValue 0x7fd1d4f7ae50>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

39.1 µs ± 96.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fd109182820>
├── 'a' --> tensor([[-0.3744,  1.5878,  3.7638],
│                   [ 0.4636,  0.2477, -0.2154],
│                   [-0.3744,  1.5878,  3.7638],
│                   [ 0.4636,  0.2477, -0.2154],
│                   [-0.3744,  1.5878,  3.7638],
│                   [ 0.4636,  0.2477, -0.2154],
│                   [-0.3744,  1.5878,  3.7638],
│                   [ 0.4636,  0.2477, -0.2154],
│                   [-0.3744,  1.5878,  3.7638],
│                   [ 0.4636,  0.2477, -0.2154],
│                   [-0.3744,  1.5878,  3.7638],
│                   [ 0.4636,  0.2477, -0.2154],
│                   [-0.3744,  1.5878,  3.7638],
│                   [ 0.4636,  0.2477, -0.2154],
│                   [-0.3744,  1.5878,  3.7638],
│                   [ 0.4636,  0.2477, -0.2154]])
└── 'x' --> <FastTreeValue 0x7fd109182760>
    └── 'c' --> tensor([[ 0.8382,  0.8111, -1.2707,  0.0985],
                        [ 0.1601, -0.2383,  0.5786,  0.0639],
                 

In [23]:
%timeit t_cat(trees)

37 µs ± 46.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

77.4 µs ± 105 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.3744,  1.5878,  3.7638],
                [ 0.4636,  0.2477, -0.2154]],
       
               [[-0.3744,  1.5878,  3.7638],
                [ 0.4636,  0.2477, -0.2154]],
       
               [[-0.3744,  1.5878,  3.7638],
                [ 0.4636,  0.2477, -0.2154]],
       
               [[-0.3744,  1.5878,  3.7638],
                [ 0.4636,  0.2477, -0.2154]],
       
               [[-0.3744,  1.5878,  3.7638],
                [ 0.4636,  0.2477, -0.2154]],
       
               [[-0.3744,  1.5878,  3.7638],
                [ 0.4636,  0.2477, -0.2154]],
       
               [[-0.3744,  1.5878,  3.7638],
                [ 0.4636,  0.2477, -0.2154]],
       
               [[-0.3744,  1.5878,  3.7638],
                [ 0.4636,  0.2477, -0.2154]]]),
    x: Batch(
           c: tensor([[[ 0.8382,  0.8111, -1.2707,  0.0985],
                       [ 0.1601, -0.2383,  0.5786,  0.0639],
                       [-1.9713,  0.9861, -2.0437, -1.0594]],
         

In [26]:
%timeit Batch.stack(batches)

100 µs ± 202 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.3744,  1.5878,  3.7638],
               [ 0.4636,  0.2477, -0.2154],
               [-0.3744,  1.5878,  3.7638],
               [ 0.4636,  0.2477, -0.2154],
               [-0.3744,  1.5878,  3.7638],
               [ 0.4636,  0.2477, -0.2154],
               [-0.3744,  1.5878,  3.7638],
               [ 0.4636,  0.2477, -0.2154],
               [-0.3744,  1.5878,  3.7638],
               [ 0.4636,  0.2477, -0.2154],
               [-0.3744,  1.5878,  3.7638],
               [ 0.4636,  0.2477, -0.2154],
               [-0.3744,  1.5878,  3.7638],
               [ 0.4636,  0.2477, -0.2154],
               [-0.3744,  1.5878,  3.7638],
               [ 0.4636,  0.2477, -0.2154]]),
    x: Batch(
           c: tensor([[ 0.8382,  0.8111, -1.2707,  0.0985],
                      [ 0.1601, -0.2383,  0.5786,  0.0639],
                      [-1.9713,  0.9861, -2.0437, -1.0594],
                      [ 0.8382,  0.8111, -1.2707,  0.0985],
                      [ 0.1601, -

In [28]:
%timeit Batch.cat(batches)

180 µs ± 146 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

424 µs ± 6.04 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
